Python program to read in the sounding profile, establish a distribution of hailstones, and send them through the melting program. Considers only falling hail, melting, and terminal velocity.  

In [1]:
import matplotlib 
import matplotlib.pyplot as plt
import numpy as np
import melt_G1969orig, melt_RH87, melt_WC2020, melt_ZL1994
import terminl_Bohm89X_Re, terminl_H2018V_D, terminl_H2018X_Re, terminl_HW2014X_Re, terminl_RH87X_Re
from metpy.calc import mixing_ratio_from_relative_humidity, potential_temperature, density, moist_lapse
from metpy.calc import lcl, parcel_profile, wind_components, cape_cin, pressure_to_height_std, height_to_pressure_std, dewpoint_from_relative_humidity
from metpy.calc import relative_humidity_from_dewpoint, wet_bulb_temperature, equivalent_potential_temperature
from metpy.units import units
from metpy.plots import SkewT
import datetime
import netCDF4 as nc4
import geocat.viz as gv
from math import pi
from scipy.signal import savgol_filter

In [2]:
# Set directories and file names

# Directories
PlotDirectory = '/Directory/Save/Figures/'

SoundingDirectory = '/Directory/Sounding/Data/'
SoundingFile = ''

HailDirectory = '/Directory/HailSize/Data/' # Used for STEPS
#HailDirectory = 'None' # Used for GRIP

# Files
HailFile = 'Name_of_HailSize_File.nc' # Used for STEPS
#HailFile = 'None' # Used for GRIP


In [4]:
# Set flags

# Behavior
UpDraftType = Constant 
#UpdraftType = Linear 
#UpdraftType = Other

SheddingFlag = True 
#SheddingFlag = False

# Height where the hail starts
HailReleaseHeight = 4.8 # Used for STEPS
#HailReleaseHeight = 5.65 # Used for GRIP

# Where sounding data from 
SoundingDataType = 'Other' # Used for STEPS 
#SoundingDataType = 'UWyoming' # Used for GRIP

# Plotting
SoundingPlotFlag = True


In [ ]:
# Set hail size distribution and vertical velocity profile, if applicable. Used for GRIP

Assumed_HailSizeDistribution_Sizes = [0]
Assumed_HailSizeDistribution_Concentrations = [0]
#Assumed_HailSizeDistribution_Sizes = np.array((300, 500, 700, 900, 1100, 1300, 1500, 1700, 2000, 2400, 2800, 3200, 3600, 4000))/10**6 # meters
#Assumed_HailSizeDistribution_Concentrations = np.array((5.9199277e4, 1.0944318e4, 3.7392256e3, 1.4063240e3, 6.0730298e2, 2.8299753e2, 
#                                                        1.5871533e2, 7.8005508e1, 7.8636177e1, 2.9596844e1, 1.5568781e1, 5.9350193e0, 
#                                                        6.3030338e0, 3.5067139e0)) #m^3

Assumed_VerticalVelocityProfile = [0]
#Assumed_VerticalVelocityProfile =  np.array([-0.02, -0.06, -.1, -.14, -.16, -0.2, -0.24, -0.28, -0.32, -0.69, -0.8, -0.94, -0.96, -0.83, -0.98, -1.05,
#                    -0.89, -0.8, -0.82, -0.77, -0.63, -0.6, -0.49, -0.55, -0.39, -0.32, -0.26, -0.19, -0.08, -0.16, -0.01,
#                    -0.09, 0.02, -0., 0.06, 0.04, 0.19, 0.13, 0.21, 0.24, 0.22, 0.28, 0.3, 0.27, 0.41, 0.47, 0.44,
#                    0.49, 0.49, 0.54, 0.45, 0.61, 0.45, 0.48, 0.58, 0.65, 0.6, 0.67, 0.66, 0.63, 0.7, 0.77, 0.78,
#                    0.8, 0.66, 0.6, 0.66, 0.65, 0.64, 0.71, 0.66, 0.74, 0.69, 0.74, 0.71, 0.85, 0.93, 0.91, 0.96,
#                    0.99, 0.91, 1., 0.9, 0.87, 0.99, 0.93, 1.06, 1.12, 1.12, 1.12, 1.38, 1.44, 1.5, 1.47, 1.54, 1.39,
#                    1.51, 1.52, 1.47, 1.65, 1.76, 2.02, 2.22, 2.32, 2.44, 2.34, 2.5, 2.55, 2.48, 2.57, 2.57,2.37, 2.47, 2.58, 2.54, 
#                    2.51, 2.41, 2.58, 2.57, 2.81, 2.7, 2.76, 2.72, 2.55, 2.43, 2.44, 2.45, 2.43, 2.43, 2.26, 2.4, 2.46, 2.51, 2.45,
#                    2.43, 2.41, 2.33, 2.25, 2.35, 2.35, 2.45, 2.42, 2.51, 2.54, 2.46, 2.57, 2.55, 2.56, 2.67, 2.71, 2.57, 2.49, 2.6,
#                    2.5, 2.67, 2.68, 3.04, 2.96, 2.98, 2.99, 3.05])
#Assumed_VerticalVelocityProfile = savgol_filter(Assumed_VerticalVelocityProfile, 15, 3, mode='nearest')


In [6]:
# Set time step and maximum number of steps

secdel = 1.0 # 1 second time step
numt = 5500 # number of times


In [5]:
# Set hail parameters

CRIT = 2.0E-4 # mass capable of being supported on the stone's surface


In [8]:
# Define general height profile, used to create vertical velocity profile

HeightProfile = np.arange(0, 10.1, 0.01)*1000 # m # Used for STEPS
#HeightProfile = np.arange(0, 8.05, 0.05)*1000 # m # Used for GRIP


In [3]:
# Set parameterization names

MeltNames = ['mG1969orig', 'mRH87', 'mWC2020', 'mZL1994']
TerminalNames = ['tBohm89xre', 'tH2018vd', 'tH2018xre', 'tHW2014xre', 'tRH87xre']


In [9]:
# Set up plotting

cmap_raw = matplotlib.colors.LinearSegmentedColormap.from_list("", ["midnightblue", "royalblue", "steelblue",
                                                                    "cadetblue", "seagreen", "olivedrab", 
                                                                    "olive", "darkgoldenrod", "sienna", 
                                                                    "brown", "firebrick", "darkmagenta",
                                                                    "indigo", "black"])

PlotColors = ['indigo', 'darkviolet', 'slateblue', 'mediumblue', 'cornflowerblue', 'green', 'limegreen', 
              'darkorange', 'goldenrod', 'brown', 'crimson', 'deeppink', 'lightcoral', 'dimgray'] 

PlotColors2 = ['mediumblue', 'forestgreen', 'darkorange', 'firebrick', 'deeppink']


In [ ]:
# Load hail data

if HailDirectory != 'None' and HailFile != 'None':
    Data = nc4.Dataset(HailDirectory + HailFile, 'r')
    
    Bins = np.array(Data.variables['Bins'])[:] #m
    #num_each_diameter = np.array(Data.variables['TotalConcentration'])[:] #m^-3
    num_each_diameter = np.array(Data.variables['MeanConcentration'])[:] #m^-3
    Diameter = np.array(Data.variables['Diameter'])[:] #m, mass weighted mean diameter
    Temperature = np.array(Data.variables['Temperature'])[:] #C
    
    Data.close()
    
    FullTimes = []
    for t in range(0, len(FlightTimes)):
        FullTimes = np.append(FullTimes, 
                              datetime.datetime(int(FlightYear), int(FlightMonth), int(FlightDay), 0, 0, 0) + 
                              datetime.timedelta(seconds=float(FlightTimes[t])))
    
    # Select bins only with non-zero numbers
    d_0 = Bins[num_each_diameter > 0]
    numd = len(d_0)
    
    num_each_diameter = num_each_diameter[num_each_diameter > 0]
    
else:
    d_0 = Assumed_HailSizeDistribution_Sizes
    numd = d_0.shape[0]
    
    num_each_diameter = Assumed_HailSizeDistribution_Concentrations


In [ ]:
# Load sounding data and plot

# Load file
if SoundingDataType == 'Other':
    SoundingData = np.genfromtxt(SoundingDirectory + SoundingFile, usecols=(1, 14, 2, 3, 8, 7), 
                                 skip_header=14, dtype=('f','f','f','f','f','f'), 
                                 names=('Pressure', 'Height', 'Temperature', 'DewPoint', 'WindDirection', 'WindSpeed'))
    
    Pressure = np.array(SoundingData['Pressure']) # hPa
    Height = np.array(SoundingData['Height']) #m
    Temperature = np.array(SoundingData['Temperature']) # C
    DewPoint = np.array(SoundingData['DewPoint']) # C
    WindDirection = np.array(SoundingData['WindDirection']) #deg
    WindSpeed = np.array(SoundingData['WindSpeed']) #knot
    
    # Omit missing data
    Keep = np.squeeze(np.array(np.where(((Pressure != 999) & ~np.isnan(Pressure) & (Temperature != 999) & (DewPoint != 999) & 
                                         (WindSpeed != 999) & (WindDirection != 999)))))
    Pressure = Pressure[Keep]
    Height = Height[Keep]
    Temperature = Temperature[Keep]
    DewPoint = DewPoint[Keep]
    WindDirection = WindDirection[Keep]
    WindSpeed = WindSpeed[Keep]
    
    Keep = np.squeeze(np.array(np.where(Pressure[1::] != Pressure[0:-1])))
    Pressure = Pressure[Keep]
    Height = Height[Keep]
    Temperature = Temperature[Keep]
    DewPoint = DewPoint[Keep]
    WindDirection = WindDirection[Keep]
    WindSpeed = WindSpeed[Keep]
    
elif SoundingDataType == 'UWyoming':
    Data = np.genfromtxt(SoundingDirectory + SoundingFile, usecols=(0, 1, 2, 3, 6, 7), skip_header=4, skip_footer=32, 
                     dtype=('f','f','f','f','f','f'), 
                     names=('Pressure', 'Height', 'Temperature', 'DewPoint', 'WindDirection', 'WindSpeed'))
   
    Pressure = np.array(Data['Pressure']) # hPa
    Temperature = np.array(Data['Temperature']) # C
    DewPoint = np.array(Data['DewPoint']) # C
    Height = np.array(Data['Height']) #m
    WindDirection = np.array(Data['WindDirection']) #deg
    WindSpeed = np.array(Data['WindSpeed']) #m/s

# Isolate Surface data
sfcP = Pressure[0]
sfcT = Temperature[0]
sfcD = DewPoint[0]

if SoundingPlotFlag == True:
    fig = plt.figure(figsize=(9, 8))
    skew = SkewT(fig)
    ax = skew.ax
    gv.set_titles_and_labels(ax, maintitle='Sounding: \n ' + SoundingFile, maintitlefontsize=16)
    skew.plot(Pressure, Temperature, 'r', linewidth=3)
    skew.plot(Pressure, DewPoint, 'g', linewidth=3)
    if f != 8:
        skew.plot_barbs(Pressure[0:-20:2], Uwind[0:-20:2], Vwind[0:-20:2])
        skew.ax.set_ylim(1000, 100)
        skew.ax.set_xlim(-45, 32)
    else:
        skew.plot_barbs(Pressure[0::75], Uwind[0::75], Vwind[0::75])
        skew.ax.set_ylim(1000, 200)
        skew.ax.set_xlim(-30, 35)
    # Plot LCL temperature as black dot
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
    # Plot the parcel profile as a black line
    skew.plot(Pressure, parcel_prof, 'k', linewidth=2, alpha=0.5)
    # Shade areas of CAPE and CIN
    skew.shade_cin(Pressure*units('hPa'), Temperature*units('degC'), parcel_prof, DewPoint*units('degC'))
    skew.shade_cape(Pressure*units('hPa'), Temperature*units('degC'), parcel_prof, alpha=0.1)
    # Plot a zero degree isotherm
    skew.ax.axvline(0, color='dimgray', linestyle=':', linewidth=4, alpha=0.5)
    skew.ax.axhline(Pressure[FlightIndex], color='dimgray', linestyle='--', linewidth=2, alpha=0.8)
    # Add the relevant special lines
    skew.plot_dry_adiabats(alpha=0.25)
    skew.plot_moist_adiabats(alpha=0.25)
    skew.plot_mixing_lines(alpha=0.25)
    # Add a secondary axis that automatically converts between pressure and height
    # assuming a standard atmosphere. The value of -0.12 puts the secondary axis
    # 0.12 normalized (0 to 1) coordinates left of the original axis.
    secax = skew.ax.secondary_yaxis(-0.15, functions=(
        lambda Pressure: pressure_to_height_std(units.Quantity(Pressure, 'hPa')).m_as('km'),
        lambda h: height_to_pressure_std(units.Quantity(h, 'km')).m))
    secax.yaxis.set_major_locator(plt.FixedLocator([0, 1, 2, 4, 6, 8, 10, 12, 14, 16]))
    secax.yaxis.set_minor_locator(plt.NullLocator())
    secax.yaxis.set_major_formatter(plt.ScalarFormatter())
    secax.set_ylabel('Height (km)')
    plt.rcParams.update({'font.size': 14})
    #plt.show()
    plt.tight_layout()
    plt.savefig(PlotDirectory + 'Sounding.png', dpi=100, format='png')
    plt.close()
    

In [ ]:
# Calculate additional environmental variables

# Calculate environmental characteristics
RelativeHumidity = relative_humidity_from_dewpoint(Temperature*units('degC'), DewPoint*units('degC'))
        
MixingRatio = mixing_ratio_from_relative_humidity(Pressure*units('hPa'), Temperature*units('degC'), RelativeHumidity)
            
rho = density(Pressure*units('hPa'), Temperature*units('degC'), MixingRatio) # density
            
ThetaE = equivalent_potential_temperature(Pressure*units('hPa'), Temperature*units('degC'), DewPoint*units('degC'))

parcel_prof = parcel_profile(Pressure*units('hPa'), sfcT*units('degC'), sfcD*units('degC'))

# Calculate additional data points
if SoundingDataType == 'UWyoming':
    Uwind, Vwind = wind_components(WindSpeed*units('knots'), WindDirection*units.deg)
elif SoundingDataType == 'Other':
    Uwind, Vwind = wind_components(WindSpeed*units('m/s'), WindDirection*units.deg)
lcl_pressure, lcl_temperature = lcl(sfcP*units('hPa'), sfcT*units('degC'), sfcD*units('degC'))

# Determine index of flight level, lcl, and hail release index
lclIndex = np.squeeze(np.array(np.nanargmin(np.absolute(np.subtract(Pressure, np.ones_like(Pressure)*np.array(lcl_pressure))))))
FlightIndex = np.array(np.nanargmin(np.absolute(np.subtract(Height, np.ones_like(Height)*np.nanmean(FlightAltitude)))))
HailReleaseIndex = np.array(np.nanargmin(np.absolute(np.subtract(Height, np.ones_like(Height)*(HailReleaseHeight*1000)))))

lcl_height = Height[lclIndex] - Height[0]
freezing_height = Height[frzlvl] - Height[0]


In [ ]:
# Calculate and plot hail environment

# Calculate temperature in cloud
temp_cloud = np.copy(parcel_prof[lclIndex:FlightIndex+1])

# Calculate mixing ratio
satq = mixing_ratio_from_relative_humidity(Pressure[lclIndex:FlightIndex+1]*units('hPa'), temp_cloud, 1.)
# mixing ratio assuming RH=1 between lcl and freezing level, merge raw mixing ratio and with 
# saturation mixing ratio between lcl and freezing level kg/kg
RLAYER = np.append(MixingRatio[0:lclIndex], satq.magnitude) 

TLAYER = np.append(Temperature[0:lclIndex]+ 273.155, temp_cloud.magnitude)
PLAYER = Pressure[0:FlightIndex+1] * 100.0  # Mean sub and in cloud pressure, hPa back to Pa
HLAYER = Height[0:FlightIndex+1]

# Determine freezing level in cloud
frzlvl = np.squeeze(np.array(np.nanargmin(np.absolute(TLAYER-273.155)))) 
freezing_height = HLAYER[frzlvl]

# Calculate dewpoint in cloud
dewpoint_cloud = dewpoint_from_relative_humidity(TLAYER*units('K'), np.append(RelativeHumidity[0:lclIndex], np.ones_like(temp_cloud)))

# Calculate hail freezing level
HailFrzIndex = np.squeeze(np.array(np.nanargmin(np.absolute(TLAYER - 273.155))))
HeightHailFrz = Height[HailFrzIndex]

# Plot environmental profile experienced by hail
if SoundingPlotFlag == True:
    fig = plt.figure(figsize=(9, 8))
    skew = SkewT(fig)
    ax = skew.ax
    gv.set_titles_and_labels(ax, maintitle='Hail Environment')
    skew.plot(PLAYER/100, TLAYER-273.155, 'r', linewidth=3)
    skew.plot(PLAYER/100, dewpoint_cloud, 'g', linewidth=3, linestyle='--')
    skew.ax.set_ylim(1000, 450)
    skew.ax.set_ylabel('Pressure [hPa]')
    skew.ax.set_xlim(0, 35)
    # Plot LCL temperature as black dot
    #skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
    # Plot a zero degree isotherm
    skew.ax.axvline(0, color='dimgray', linestyle=':', linewidth=4, alpha=0.5)
    skew.ax.axhline(Pressure[FlightIndex], color='dimgray', linestyle='--', linewidth=2, alpha=0.8)
    # Add the relevant special lines
    skew.plot_dry_adiabats(alpha=0.25)
    skew.plot_moist_adiabats(alpha=0.25)
    skew.plot_mixing_lines(alpha=0.25)
    # Add a secondary axis that automatically converts between pressure and height
    # assuming a standard atmosphere. The value of -0.12 puts the secondary axis
    # 0.12 normalized (0 to 1) coordinates left of the original axis.
    secax = skew.ax.secondary_yaxis(-0.15, functions=(
        lambda Pressure: pressure_to_height_std(units.Quantity(Pressure, 'hPa')).m_as('km'),
        lambda h: height_to_pressure_std(units.Quantity(h, 'km')).m))
    secax.yaxis.set_major_locator(plt.FixedLocator([0, 1, 2, 4, 6, 8, 10, 12, 14, 16]))
    secax.yaxis.set_minor_locator(plt.NullLocator())
    secax.yaxis.set_major_formatter(plt.ScalarFormatter())
    secax.set_ylabel('Height (km)')
    #plt.show()
    plt.tight_layout()
    plt.savefig(PlotDirectory + 'HailSounding.png', dpi=100, format='png')
    plt.close()

# Change names/units to what terminl, melt are expecting, and subset to below freezing lvl
zh = Height
zh = zh[0:FlightIndex+1] # height above ground, start at flight level
zh = zh - zh[0]

DENSA = rho[0:FlightIndex+1].magnitude # in cloud density
DENSE = 917. # Hail density [kg m-3]
    

In [ ]:
# Define updraft

if UpDraftType == 'Constant':
    # Create constant updraft
    Index = np.squeeze(np.array(np.nanargmin(np.absolute(np.subtract(
        HeightProfile, np.ones_like(HeightProfile)*lcl_height)))))
    Updraft = np.zeros_like(HeightProfile)
    Updraft[Index::] = 5

elif UpDraftType == 'Linear':
    Index = np.squeeze(np.array(np.nanargmin(np.absolute(np.subtract(HeightProfile, np.ones_like(HeightProfile)*lcl_height)))))
    Updraft = np.zeros_like(HeightProfile)
    Updraft[Index::] = np.nanmean(VerticalVelocity[VerticalVelocity > 0]) 

elif UpdraftType == 'Other':
    Updraft = Assumed_VerticalVelocityProfile


The next section is the HAILMELT code. During each timestep during each flight:
   1) calculate terminal velocity
   2) calculate distance hailstone has fallen
   3) interpolate temp, density, pressure, and qv at new height
   5) calculate how much has melted over that distance and new diameter
   6) Repeat until hail is swept upward, melts, hits the ground, or the time limit is reached

- This process is done for each parameterization combination

In [11]:
# HAILCAST

# Initialize hail tracking array
alltv = np.ones((4, 5, numd, numt), dtype=float)*np.nan # all terminal velocities
alld = np.ones((4, 5, numd, numt), dtype=float)*np.nan # all diameters
allz = np.ones((4, 5, numd, numt), dtype=float)*np.nan # all heights
Up = np.ones((4, 5, 2, numd), dtype=float)*np.nan
Gone = np.ones((4, 5, 2, numd), dtype=float)*np.nan

# Run HAILCAST, looping through parameterizations
for m in range(0, 4):     
    for t in range(0, 5): 
        print(m, t, MeltNames[m], TerminalNames[t])

        # Find properties of initial hailstone location, assuming starting at freezing level 
        z = zh[-1] # height above ground
        p = PLAYER[-1]
        d_in = d_0
        
        # Initial terminal velocity calculation
        if t == 0: 
            tv = [terminl_Bohm89X_Re.terminl_bohm89x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in] 
        elif t == 1:
            tv = [terminl_H2018V_D.terminl_h2018v_d(d, p) for d in d_in]
        elif t == 2:
            tv = [terminl_H2018X_Re.terminl_h2018x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in]
        elif t == 3:
            tv = [terminl_HW2014X_Re.terminl_hw2014x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in]   
        elif t == 4: 
            tv = [terminl_RH87X_Re.terminl_rh87x_re(DENSA[-1], DENSE, d, TLAYER[-1]) for d in d_in]
        tv = np.array(tv)
                            
        MeltMass = np.zeros((numd), dtype=float)

        for n in np.arange(numt): # loop through times  
            #calculate distance hailstone has fallen
            ldepth = tv*secdel # depth fell
            z = z - ldepth # updated height

            d_in[z <= 0] = np.nan
            z[z <= 0] = np.nan

            allz[m, t, :, n] = z
            if np.all(z<z[0]) or np.all(np.isnan(z)): # Break out of loop if all stones have hit the ground
                break

            #interpolate new temp, density, pressure at new height
            NEWTK = np.interp(z, zh, TLAYER) 
            NEWP = np.interp(z, zh, PLAYER)
            NEWR = np.interp(z, zh, RLAYER)
            NEWDENSA = np.interp(z, zh, DENSA)

            ## Melt ##
            # Determine if warm enough to melt. If warm enough melt, change mass, determine water fraction
            newd = np.ones_like(d_in)*np.nan
            for d in np.arange(numd):
                # Determine starting mass
                Mass_Original = (4/3)*pi*(d_in[d]/2)**3 * DENSE
                    
                if m == 0: 
                    if NEWTK[d] > 273.155:
                        # Warm enough, melt hail
                        newd[d] = melt_G1969orig.melt_g1969orig(d_in[d], NEWTK[d], NEWP[d], NEWR[d], 
                                                                ldepth[d], tv[d])
                        # Determine if hail still exists
                        if newd[d] < 1e-4 or np.isnan(newd[d]):
                            # Hail gone
                            newd[d] = np.nan
                            Gone[m, t, 0, d] = d_in[d]
                            Gone[m, t, 1, d] = z[d]
                        else:
                            # Calculate mass change and water fraction
                            Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                            MeltMass_Step = Mass_Original - Mass_New
                            MeltMass[d] = MeltMass[d] + MeltMass_Step
                            
                    elif NEWTK[d] <= 273.155:
                        # Too cold, hail remains same size
                        newd[d] = d_in[d]
                    
                elif m == 1: 
                    if NEWTK[d] > 273.155:
                        # Warm enough, melt hail
                        newd[d] = melt_RH87.melt_rh87orig(d_in[d], NEWTK[d], NEWP[d], NEWR[d], tv[d], secdel, DENSE, DENSA)
                        # Determine if hail still exists
                        if newd[d] < 1e-4 or np.isnan(newd[d]):
                            # Hail gone
                            newd[d] = np.nan
                            Gone[m, t, 0, d] = d_in[d]
                            Gone[m, t, 1, d] = z[d]
                        else:
                            # Calculate mass change and water fraction
                            Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                            MeltMass_Step = Mass_Original - Mass_New
                            MeltMass[d] = MeltMass[d] + MeltMass_Step
                            
                    elif NEWTK[d] <= 273.155:
                        # Too cold, hail remains same size
                        newd[d] = d_in[d]
                        
                elif m == 2: 
                    if NEWTK[d] > 273.155:
                        # Warm enough, melt hail
                        newd[d] = melt_WC2020.melt_wc2020short(d_in[d], NEWTK[d], NEWP[d], NEWR[d], tv[d], secdel, DENSE, DENSA)
                        # Determine if hail still exists
                        if newd[d] < 1e-4 or np.isnan(newd[d]):
                            # Hail gone
                            newd[d] = np.nan
                            Gone[m, t, 0, d] = d_in[d]
                            Gone[m, t, 1, d] = z[d]
                        else:
                            # Calculate mass change and water fraction
                            Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                            MeltMass_Step = Mass_Original - Mass_New
                            MeltMass[d] = MeltMass[d] + MeltMass_Step
                            
                    elif NEWTK[d] <= 273.155:
                        # Too cold, hail remains same size
                        newd[d] = d_in[d]
                        
                elif m == 3: 
                    if NEWTK[d] > 273.155:
                        # Warm enough, melt hail
                        newd[d] = melt_ZL1994.melt_zl1994sphere(d_in[d], NEWTK[d], NEWP[d], NEWR[d], tv[d], secdel, DENSE, DENSA)
                        # Determine if hail still exists
                        if newd[d] < 1e-4 or np.isnan(newd[d]):
                            # Hail gone
                            newd[d] = np.nan
                            Gone[m, t, 0, d] = d_in[d]
                            Gone[m, t, 1, d] = z[d]
                        else:
                            # Calculate mass change and water fraction
                            Mass_New = (4/3)*pi*(newd[d]/2)**3 * DENSE                        
                            MeltMass_Step = Mass_Original - Mass_New
                            MeltMass[d] = MeltMass[d] + MeltMass_Step
                            
                    elif NEWTK[d] <= 273.155:
                        # Too cold, hail remains same size
                        newd[d] = d_in[d] 

                ## Shedding ##
                if SheddingFlag == True:
                    # Only shed if hail exists and above freezing
                    if newd[d] > 0 or ~np.isnan(newd[d]) and NEWTK[d] > 273.15:
                        # If the mass of the melted water on the hail is above a critical limit, shed water
                        if MeltMass[d] > CRIT:
                            MeltMass[d] = np.copy(CRIT)

            # Save melt, raindrop, and hail diameter data. 
            alld[m, t, :, n] = np.copy(newd) # Save diameters
            
            # Set hail diameters for next time step
            d_in = np.array(newd) # Update diameters

            ## Terminal velocity ##
            # These are terminal velocities for the "new hail"
            # Negative tv indicates that the hail is swept upward
            if t == 0: 
                tv = [terminl_Bohm89X_Re.terminl_bohm89x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)] 
            elif t == 1:
                tv = [terminl_H2018V_D.terminl_h2018v_d(newd[d], NEWP[d]) for d in np.arange(numd)]          
            elif t == 2:
                tv = [terminl_H2018X_Re.terminl_h2018x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)]              
            elif t == 3:
                tv = [terminl_HW2014X_Re.terminl_hw2014x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)]               
            elif t == 4: 
                tv = [terminl_RH87X_Re.terminl_rh87x_re(NEWDENSA[d], DENSE, newd[d], NEWTK[d]) for d in np.arange(numd)]
            tv = np.array(tv)

            # Correct terminal velocity for impact of updraft velocity
            for d in range(0, numd):
                if ~np.isnan(newd[d]) and ~np.isnan(newd[d]):
                    # Determine where the hail is in the height profile
                    Index = np.squeeze(np.array(np.nanargmin(np.absolute(np.subtract(HeightProfile, 
                                                                                     np.ones_like(HeightProfile)
                                                                                     *(z[d]))))))
                    if Updraft[Index] < tv[d]:
                        # Terminal velocity is stronger than updraft. 
                        # Reduce terminal velocity by removing updraft component
                        tv[d] = tv[d] - Updraft[Index]
                    else:
                        # Terminal velocity is weaker than updraft. Hail is swept up.
                        # Stop tracking hail 
                        tv[d] = np.nan
                        Up[m, t, 0, d] = d_in[d]
                        Up[m, t, 1, d] = z[d]

            # Remove data if hail melts or is swept upward
            d_in[~np.isnan(Gone[m, t, 0, :])] = np.nan
            d_in[~np.isnan(Up[m, t, 0, :])] = np.nan
            z[~np.isnan(Gone[m, t, 0, :])] = np.nan
            z[~np.isnan(Up[m, t, 0, :])] = np.nan

            # Save terminal velocity data
            alltv[m, t, :, n] = np.copy(tv)
            
        # Plot Results for individual parameterization pair
        # Time series
        fig = plt.figure(figsize=(10, 8))
        plt.suptitle('Terminal: ' + TerminalNames[t] + ', Melt: ' + MeltNames[m] + ': Time Series', size=14, y=0.98 )

        ax1 = plt.subplot(211)
        ax1.set_title('Height vs Time', size=14)
        for i in range(0, numd):
            ax1.plot(np.arange(0, numt, 1), allz[m, t, i, :]/1000, linewidth=3, color=PlotColors[i], label=str(d_0[i])+' mm')
        plt.ylim(bottom=0)
        ax1.set_ylabel('Height [km]', size=14)
        ax1.set_xlim(0, numt)
        ax1.tick_params(axis='x', labelbottom = False)
        ax1.tick_params(axis='both', labelsize=12)
        ax1.grid(True)

        ax2 = plt.subplot(212)
        ax2.set_title('Diameter vs Time', size=14)
        for i in range(0, numd):
            ax2.plot(np.arange(0, numt, 1), alld[m ,t, i, :]*1.E3, linewidth=3, color=PlotColors[i], label=str(d_0[i])+' mm')
        ax2.set_ylim(0, np.nanmax(alld[m, t, :, :])*1.E3)
        ax2.set_ylabel('Diameter [mm]', size=14)
        ax2.set_xlim(0, numt)
        ax2.tick_params(axis='x', labelbottom = False)
        ax2.tick_params(axis='both', labelsize=12)
        ax2.grid(True)

        plt.tight_layout()
        plt.savefig(PlotDirectory + TerminalNames[t] + '-' + MeltNames[m] + '_Height-Diameter-TimeSeries.png', dpi=100, format='png')
        plt.close()

        # Hail size - diameter distribution
        plt.figure(figsize=(10, 6))
        plt.title('Terminal: ' + TerminalNames[t] + ', Melt: ' + MeltNames[m] + ' \n Time Series', size=14, y=1.01)
        for i in np.arange(numd):
            #plt.plot(alld[m, t, i,:]*1.E3, allz[m, t, i,:]/1000, color=colors[i])
            C = plt.scatter(alld[m, t, i,:]*1.E3, allz[m, t, i,:]/1000, s=20, c=np.arange(0, numt, 1), cmap='turbo', vmin=0, vmax=numt, alpha=0.6)
        cbar = plt.colorbar(C, pad=0.01)
        plt.scatter(Gone[0, :]*1.E3, Gone[1, :]/1000, c='black', marker='o', s=80)
        plt.scatter(Up[0, :]*1.E3, Up[1, :]/1000, c='black', marker='x', s=80)
        plt.plot([0, d_0[-1]*1.E3+2], [lcl_height/1000, lcl_height/1000], color='black', linestyle='--', alpha=0.6)
        plt.plot([0, d_0[-1]*1.E3+2], [freezing_height/1000, freezing_height/1000], color='black', linestyle=':', linewidth=3, alpha=0.6)
        cbar.set_label('Time Step [s]', fontsize=14)
        #plt.scatter(Up[0, :], Up[1, :], c='black', marker='s')
        plt.ylim(bottom=0)
        plt.ylabel('Height [km]', size=14)
        #plt.xticks(Bins*1.E3, Bins*1.E3, rotation=25)
        plt.xlim(0, d_0[-1]*1.E3+2)
        plt.xlabel('Diameter [mm]', size=14)
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(PlotDirectory + TerminalNames[t] + '-' + MeltNames[m] + '_HeightD-DSD.png', dpi=100, format='png')
        plt.close()

# Save data in netcdf file
ncfile = nc4.Dataset(DataDirectory + SaveName1 + SaveName2 + '_HAILMELT-output.nc', mode='w', format='NETCDF4')
ncfile.description = 'terminal velocity indices: 1=tBohm89, 2=tH2018_vd, 3=tH2018_xre, 4=tHW2014_xre, 5=tRH87xre, melt indices: 1=mG1969, 1=mRH87, 2=WC2020, 3=RL1994'

hail_diameter_dim = ncfile.createDimension('hail_diameter_dim', numd)
sounding_dim = ncfile.createDimension('sounding_dim', HailFrzIndex+2)
time_dim = ncfile.createDimension('time_dim', MaxTimeStep)
melt_dim = ncfile.createDimension('melt_dim', 4)
terminal_dim = ncfile.createDimension('terminal_dim', 5)

HailSounding_Temperature = ncfile.createVariable('HailSounding_Temperature', np.float32, ('sounding_dim'))
HailSounding_Temperature.units = 'degC'
HailSounding_Temperature.long_name = 'Temperature profile of the modified environment the hail is falling through'

HailSounding_Dewpoint = ncfile.createVariable('HailSounding_Dewpoint', np.float32, ('sounding_dim'))
HailSounding_Dewpoint.units = 'degC'
HailSounding_Dewpoint.long_name = 'Dew point profile of the modified environment the hail is falling through'

HailSounding_Height = ncfile.createVariable('HailSounding_Height', np.float32, ('sounding_dim'))
HailSounding_Height.units = 'km'
HailSounding_Height.long_name = 'Height profile of the modified environment the hail is falling through'

HailSounding_Pressure = ncfile.createVariable('HailSounding_Pressure', np.float32, ('sounding_dim'))
HailSounding_Pressure.units = 'hPa'
HailSounding_Pressure.long_name = 'Pressure profile of the modified environment the hail is falling through'

HailSize = ncfile.createVariable('HailSize', np.float32, ('hail_diameter_dim',))
HailSize.units = 'mm'
HailSize.long_name = 'Initial size of hail, based on Ku-Z relationship'

TimeSteps = ncfile.createVariable('TimeSteps', np.int32, ('time_dim',))
TimeSteps.units = 's'
TimeSteps.long_name = 'Model timestep, maximum of all parameterization combinations and sizes'

HailDiameterTimeSeries = ncfile.createVariable('HailDiameterTimeSeries', np.float32, ('melt_dim', 'terminal_dim', 'hail_diameter_dim','time_dim'))
HailDiameterTimeSeries.units = 'mm'
HailDiameterTimeSeries.long_name = 'Diameter of hail'

HailHeightTimeSeries = ncfile.createVariable('HailHeightTimeSeries', np.float32, ('melt_dim', 'terminal_dim', 'hail_diameter_dim', 'time_dim'))
HailHeightTimeSeries.units = 'km'
HailHeightTimeSeries.long_name = 'Height of hail'

HailSounding_Temperature[:] = TLAYER - 273.155
HailSounding_Dewpoint[:] = np.array(dewpoint_cloud)
HailSounding_Height[:] = HLAYER
HailSounding_Pressure[:] = PLAYER/100

HailSize[:] = d_0[:]*1e3

TimeSteps[:] = np.array(np.arange(MaxTimeStep))

HailDiameterTimeSeries[:, :, :, :] = alld[:, :, :, :]*1.E3
HailHeightTimeSeries[:, :, :, :] = allz[:, :, :, :]/1000

ncfile.close()


658
KOUN
KOUN_19950517_18Z.txt
0 0 mG1969orig tBohm89xre
0 1 mG1969orig tH2018vd
0 2 mG1969orig tH2018xre
0 3 mG1969orig tHW2014xre
0 4 mG1969orig tRH87xre
1 0 mRH87 tBohm89xre
1 1 mRH87 tH2018vd
1 2 mRH87 tH2018xre
1 3 mRH87 tHW2014xre
1 4 mRH87 tRH87xre
2 0 mWC2020 tBohm89xre
2 1 mWC2020 tH2018vd
2 2 mWC2020 tH2018xre
2 3 mWC2020 tHW2014xre
2 4 mWC2020 tRH87xre
3 0 mZL1994 tBohm89xre
3 1 mZL1994 tH2018vd
3 2 mZL1994 tH2018xre
3 3 mZL1994 tHW2014xre
3 4 mZL1994 tRH87xre
667
KLBF
KLBF_19950621_00Z.txt
0 0 mG1969orig tBohm89xre
0 1 mG1969orig tH2018vd
0 2 mG1969orig tH2018xre
0 3 mG1969orig tHW2014xre
0 4 mG1969orig tRH87xre
1 0 mRH87 tBohm89xre
1 1 mRH87 tH2018vd
1 2 mRH87 tH2018xre
1 3 mRH87 tHW2014xre
1 4 mRH87 tRH87xre
2 0 mWC2020 tBohm89xre
2 1 mWC2020 tH2018vd
2 2 mWC2020 tH2018xre
2 3 mWC2020 tHW2014xre
2 4 mWC2020 tRH87xre
3 0 mZL1994 tBohm89xre
3 1 mZL1994 tH2018vd
3 2 mZL1994 tH2018xre
3 3 mZL1994 tHW2014xre
3 4 mZL1994 tRH87xre
668
KLBF
KLBF_19950623_00Z.txt
0 0 mG1969orig tB

/var/folders/cb/zhyy0qr14kg35txh2x2n3hvm0002n2/T/ipykernel_18624/747642594.py:213: UserWarning: Input over 12.566370614359172 radians. Ensure proper units are given.
  Uwind, Vwind = wind_components(WindSpeed*units('knots'), WindDirection*units.deg)


0 0 mG1969orig tBohm89xre
0 1 mG1969orig tH2018vd
0 2 mG1969orig tH2018xre
0 3 mG1969orig tHW2014xre
0 4 mG1969orig tRH87xre
1 0 mRH87 tBohm89xre
1 1 mRH87 tH2018vd
1 2 mRH87 tH2018xre
1 3 mRH87 tHW2014xre
1 4 mRH87 tRH87xre
2 0 mWC2020 tBohm89xre
2 1 mWC2020 tH2018vd
2 2 mWC2020 tH2018xre
2 3 mWC2020 tHW2014xre
2 4 mWC2020 tRH87xre
3 0 mZL1994 tBohm89xre
3 1 mZL1994 tH2018vd
3 2 mZL1994 tH2018xre
3 3 mZL1994 tHW2014xre
3 4 mZL1994 tRH87xre
803
KSHV
KSHV_20030602_00Z.txt
0 0 mG1969orig tBohm89xre
0 1 mG1969orig tH2018vd
0 2 mG1969orig tH2018xre
0 3 mG1969orig tHW2014xre
0 4 mG1969orig tRH87xre
1 0 mRH87 tBohm89xre
1 1 mRH87 tH2018vd
1 2 mRH87 tH2018xre
1 3 mRH87 tHW2014xre
1 4 mRH87 tRH87xre
2 0 mWC2020 tBohm89xre
2 1 mWC2020 tH2018vd
2 2 mWC2020 tH2018xre
2 3 mWC2020 tHW2014xre
2 4 mWC2020 tRH87xre
3 0 mZL1994 tBohm89xre
3 1 mZL1994 tH2018vd
3 2 mZL1994 tH2018xre
3 3 mZL1994 tHW2014xre
3 4 mZL1994 tRH87xre
819
KLBF
KLBF_20030730_00Z.txt
0 0 mG1969orig tBohm89xre
0 1 mG1969orig tH2018v